In [1]:
%load_ext lab_black

In [2]:
import json
import requests
import yaml

url = "https://ckan.opendata.swiss/api/3/action/package_show?id=haushaltsaquivalenz-einkommen-steuerpflichtiger-naturlicher-personen-nach-haushaltstyp-stadtqua"

resp = requests.get(url)
resp.content[:500]

b'{"help": "https://ckan.opendata.swiss/api/3/action/help_show?name=package_show", "success": true, "result": {"license_title": null, "maintainer": "Open Data Z\\u00fcrich", "issued": "2022-11-01T00:00:00", "title_for_slug": "haushaltsaquivalenz-einkommen-steuerpflichtiger-naturlicher-personen-nach-haushaltstyp-stadtqua", "qualified_relations": [], "private": false, "maintainer_email": "opendata@zuerich.ch", "num_tags": 7, "contact_points": [{"email": "opendata@zuerich.ch", "name": "Open Data Z\\u00'

In [3]:
resp.status_code

200

In [4]:
resp.encoding

'utf-8'

In [25]:
json_household_income = resp.json()
household_income_url = json_household_income["result"]["resources"][0]["url"]

In [26]:
import pandas as pd

zurich_household_income = pd.read_csv(household_income_url)

In [42]:
zurich_household_income_ytd = zurich_household_income.loc[
    zurich_household_income["StichtagDatJahr"]
    == zurich_household_income["StichtagDatJahr"].max()
]
zurich_household_income_ytd = zurich_household_income_ytd.dropna()
zurich_household_income_ytd.head(10)

# Tax year (technically: key date date year)	Year for which the tax was claimed (integer).
# City quarter (sort) (technically: QuarSort)	ID for the city district at the current residential address (integer).
# City quarter (code) (technical: QuarCd)	ID for the city district at the current residential address (string).
# City Quarter (Lang) (technically: QuarLang)	City district name of the current residential address (string).
# Household form sort (technically: HHtypSort)	Household form sorting code (integer).
# Household type code (technical: HHtypCd)	Household form sorting code (integer).
# Household type (technically: HHtypLang)	Household form description (string).
# Median value (technical: aequEK_p50)	Median value of the household equivalent income of natural persons in Zurich. In thousand francs (integers).
# P25 value (technical: aequEK_p25)	25% quantile value of the household equivalent income of natural persons in Zurich. In thousand francs (integers).
# P75 value (technical: aequEK_p75)	75% quantile value of the household equivalent income of natural persons in Zurich. In thousand francs (integers).

# https://data.stadt-zuerich.ch/dataset/fd_median_hhaequeinkommen_quartier_od100a

,StichtagDatJahr,QuarSort,QuarCd,QuarLang,HHtypSort,HHtypCd,HHtypLang,aequEK_p50,aequEK_p25,aequEK_p75
2676,2020,11,11,Rathaus,7,1EH,Ein-Elternhaushalt,44.93,20.31,63.23
2677,2020,11,11,Rathaus,8,EHK,Ehepaar mit Kind(ern),64.29,41.84,92.92
2678,2020,11,11,Rathaus,5,EH,Ehepaar ohne Kind(er),75.73,45.53,130.67
2679,2020,11,11,Rathaus,4,2P,Zweipersonenhaushalt,81.87,50.73,110.13
2680,2020,11,11,Rathaus,1,1P,Einpersonenhaushalt,58.30,28.60,100.15
2689,2020,12,12,Hochschulen,1,1P,Einpersonenhaushalt,58.40,29.30,107.50
2696,2020,13,13,Lindenhof,8,EHK,Ehepaar mit Kind(ern),44.12,29.29,83.22
2697,2020,13,13,Lindenhof,5,EH,Ehepaar ohne Kind(er),76.27,51.00,121.27
2698,2020,13,13,Lindenhof,4,2P,Zweipersonenhaushalt,90.07,59.80,139.47
2699,2020,13,13,Lindenhof,1,1P,Einpersonenhaushalt,67.10,33.70,107.90


In [45]:
zurich_household_income_ytd_median = (
    zurich_household_income_ytd.groupby(["QuarLang", "HHtypLang"])
    .agg("median")
    .reset_index()
)

household_dict = {
    "Ehepaar mit Kind(ern)": "Married couple with child(ren)",
    "Ehepaar ohne Kind(er)": "Married couple without child(ren)",
    "Ein-Elternhaushalt": "One-parent household",
    "Einpersonenhaushalt": "One-person household",
    "Paar mit Kind(ern)": "Couple with child(ren)",
    "Patchwork-Haushalt": "Patchwork household",
    "Wohngemeinschaft mit Kind(ern)": "Shared apartment with child(ren)",
    "Wohngemeinschaft ohne Kind(er)": "Shared apartment without child(ren)",
    "Zweipersonenhaushalt": "Two-person household",
    "Eingetragenes Paar ohne Kind(er)": "Registered couple without child(ren)",
}

zurich_household_income_ytd_median = zurich_household_income_ytd_median.replace(
    {"HHtypLang": household_dict}
)
zurich_household_income_ytd_median

,QuarLang,HHtypLang,StichtagDatJahr,QuarSort,QuarCd,HHtypSort,aequEK_p50,aequEK_p25,aequEK_p75
0,Affoltern,Married couple with child(ren),2020.0,111.0,111.0,8.0,39.24,24.52,56.14
1,Affoltern,Married couple without child(ren),2020.0,111.0,111.0,5.0,52.00,37.27,70.93
2,Affoltern,One-parent household,2020.0,111.0,111.0,7.0,30.00,15.07,46.53
3,Affoltern,One-person household,2020.0,111.0,111.0,1.0,45.20,24.90,65.20
4,Affoltern,Couple with child(ren),2020.0,111.0,111.0,10.0,42.22,32.33,59.33
...,...,...,...,...,...,...,...,...,...
266,Wollishofen,One-person household,2020.0,21.0,21.0,1.0,53.55,32.70,82.15
267,Wollishofen,Couple with child(ren),2020.0,21.0,21.0,10.0,47.28,32.67,69.08
268,Wollishofen,Shared apartment with child(ren),2020.0,21.0,21.0,13.0,53.05,32.33,82.19
269,Wollishofen,Shared apartment without child(ren),2020.0,21.0,21.0,12.0,66.03,44.44,101.45
